# MNIST

PyTorch 实践


In [2]:
import os

import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using { device } device")

Using cpu device


In [3]:
import numpy as np
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from PIL import Image

# 构建自己的数据集
class MnistDataset(Dataset):
  def __init__(self, transform=None, path=None):
    self.path = path
    self.data = os.listdir(self.path)
    self.transform = transform
    self.len = len(self.data)

  def __getitem__(self, index):
    image_index = self.data[index]
    img_path = os.path.join(self.path, image_index)
    img = Image.open(img_path)
    if self.transform:
      img = self.transform(img)

    label = int(image_index[-5])
    # label = self.oneHot(label)
    return img, label

  def __len__(self):
    return self.len

  # 将标签转为onehot编码
  def oneHot(self, label):
    tem = np.zeros(10)
    tem[label] = 1
    return torch.from_numpy(tem)

In [10]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
# 定义网络
class NeuralNetwork(nn.Module):
  # def __init__(self):
  #   super(NeuralNetwork, self).__init__()
  #   self.flatten = nn.Flatten()
  #   self.linear_relu_stack = nn.Sequential(
  #     nn.Linear(in_features=28 * 28, out_features=512),
  #     nn.ReLU(),
  #     nn.Linear(in_features=512, out_features=512),
  #     nn.ReLU(),
  #     nn.Linear(in_features=512, out_features=10),
  #   )
  # def forward(self, x):
  #   x = self.flatten(x)
  #   logits = self.linear_relu_stack(x)
  #   return logits

  def __init__(self):
    super(NeuralNetwork, self).__init__()
    self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
    self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
    self.conv2_drop = nn.Dropout2d()
    self.fc1 = nn.Linear(320, 50)
    self.fc2 = nn.Linear(50, 10)

  def forward(self, x):
    x = F.relu(F.max_pool2d(self.conv1(x), 2))
    x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
    x = x.view(-1, 320)
    x = F.relu(self.fc1(x))
    x = F.dropout(x, training=self.training)
    x = self.fc2(x)
    return F.log_softmax(x)

In [5]:
# 优化模型参数
def train_loop(dataloader, model, loss_fn, optimizer, device):
  size = len(dataloader.dataset)
  for batch, (X, y) in enumerate(dataloader):
    X = X.to(device)
    y = y.to(device)
    # 前向传播，计算预测值
    pred = model(X)
    # 计算损失
    loss = loss_fn(pred, y)
    # 反向传播，优化参数
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if batch % 100 == 0:
      loss, current = loss.item(), batch * len(X)
      print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

# 测试模型性能
def test_loop(dataloader, model, loss_fn, device):
  size = len(dataloader.dataset)
  num_batches = len(dataloader)
  test_loss, correct = 0, 0

  with torch.no_grad():
    for X, y in dataloader:
      X = X.to(device)
      y = y.to(device)
      # 前向传播，计算预测值
      pred = model(X)
      
      # 计算损失
      test_loss += loss_fn(pred, y).item()
      # 计算准确率
      correct += (pred.argmax(1) == y).type(torch.float).sum().item()

  test_loss /= num_batches
  correct /= size
  print(f"Test: \n Accuracy: {(100 * correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [11]:
from torchvision.transforms import ToTensor

if __name__ == '__main__':
  device = "cuda" if torch.cuda.is_available() else "cpu"
  print(f"Using {device} device")
  # 定义模型
  model = NeuralNetwork().to(device)
  # 设置超参数
  learning_rate = 1e-3
  batch_size = 64
  epochs = 50

  train_dataset = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()  # 对样本数据进行处理，转换为张量数据
  )
  # 测试数据集
  test_dataset = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()  # 对样本数据进行处理，转换为张量数据
  )

  train_data = DataLoader(
    dataset=train_dataset,
    # 设置批量大小
    batch_size=64,
    # 打乱样本的顺序
    shuffle=True)
  # 测试数据加载器
  test_data = DataLoader(
    dataset=test_dataset,
    batch_size=64,
    shuffle=True)

  # 加载数据集
  # train_dataset = MnistDataset(transform=ToTensor(), path='./mnist/train')
  # train_data = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

  # test_dataset = MnistDataset(transform=ToTensor(), path='./mnist/test')
  # test_data = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

  # 定义损失函数和优化器
  loss_fn = nn.CrossEntropyLoss()
  optimizer = torch.optim.SGD(params=model.parameters(), lr=learning_rate)
  # 训练模型
  for t in range(epochs):
    print(f"Epoch {t + 1}\n-------------------------------")
    train_loop(train_data, model, loss_fn, optimizer, device)
    test_loop(test_data, model, loss_fn, device)
  print("Done!")
  # 保存模型
  torch.save(model.state_dict(), 'model_weights.pth')

Using cpu device
Epoch 1
-------------------------------


/home/jiji/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:36: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


loss: 2.297560  [    0/60000]
loss: 2.301487  [ 6400/60000]
loss: 2.307737  [12800/60000]
loss: 2.301557  [19200/60000]
loss: 2.319473  [25600/60000]
loss: 2.308531  [32000/60000]
loss: 2.291257  [38400/60000]
loss: 2.296378  [44800/60000]
loss: 2.292051  [51200/60000]
loss: 2.306219  [57600/60000]
Test: 
 Accuracy: 12.0%, Avg loss: 2.296868 

Epoch 2
-------------------------------
loss: 2.291897  [    0/60000]
loss: 2.305747  [ 6400/60000]
loss: 2.299816  [12800/60000]
loss: 2.290524  [19200/60000]
loss: 2.300469  [25600/60000]
loss: 2.297340  [32000/60000]
loss: 2.308899  [38400/60000]
loss: 2.284355  [44800/60000]
loss: 2.280794  [51200/60000]
loss: 2.303311  [57600/60000]
Test: 
 Accuracy: 14.6%, Avg loss: 2.287034 

Epoch 3
-------------------------------
loss: 2.283633  [    0/60000]
loss: 2.268251  [ 6400/60000]
loss: 2.292315  [12800/60000]
loss: 2.268906  [19200/60000]
loss: 2.288314  [25600/60000]
loss: 2.303162  [32000/60000]
loss: 2.279492  [38400/60000]
loss: 2.255013  [4

KeyboardInterrupt: 